In [1]:
import requests as rs
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

In [9]:
auto_data = {"Marka" : [],             # brand
                 
             "Model" : [],              # models
                 
            "Buraxılış ili": [],         # year
             
             "Ban növü" : [],            # vehicle type
                  
            "Rəng": [],                  # color
                 
             "Mühərrik" :[],             # fuel-type
                 
            "Yürüş" :[],                # kilometer
                 
            "Sürətlər qutusu" :[],       # gearbox  
                 
            "Yeni" :[],                 # new or used
                 
            "Vəziyyəti":[],             # whether car has any damage
                 
            "Qiymət" :[],               # price
                  
            "Link" :[] }                # advertisement link

page_num=1
    
while page_num <=30:  # to access all pages on the web site
    
   
    
    url = f'https://turbo.az/autos?page/{page_num}/'  
    header= {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}
    r = rs.get(url, headers=header)


    if r.status_code == 200:
        soup = bs(r.content, 'lxml')
        
        # extract auto links from webpage
        auto_links= soup.find_all('a', {'class':"products-i__link"}, href=True)
        
        for i in range(len(auto_links)-1):
            link = 'https://turbo.az' + auto_links[i].get('href')
            auto_data['Link'].append(link)
            
            r1 = rs.get(link, headers=header) 
            soup1 = bs(r1.content, 'lxml')
            
            # price
            qiymet=soup1.find('div', {'class':'product-price__i product-price__i--bold'}).text
            
            auto_data['Qiymət'].append(qiymet)
            
            properties = soup1.find_all('label', {'class':"product-properties__i-name"})
                                
            # extract properties that we want 
            for i in range(len(properties)-1):
                if properties[i].text in ['Marka', 'Model', 'Buraxılış ili', "Ban növü", 'Rəng', 'Mühərrik', 'Yürüş', 
                                      'Sürətlər qutusu','Yeni', 'Vəziyyəti'] :
                    auto_data[properties[i].text].append(properties[i].next_sibling.text)         
             
                
    else: 
        print('Page not found')
        
    page_num+=1
            
            
                
                

In [10]:
# If the property that we want is not available, fills it with NaN in dataframe.
max_length = max(len(v) for v in auto_data.values())

for key, value in auto_data.items():
    if len(value) < max_length:
        diff = max_length - len(value)
        auto_data[key] = value + [np.nan] * diff

In [11]:
auto_data =pd.DataFrame(auto_data)

In [12]:
auto_data.head(10)

,Marka,Model,Buraxılış ili,Ban növü,Rəng,Mühərrik,Yürüş,Sürətlər qutusu,Yeni,Vəziyyəti,Qiymət,Link
0,Nissan,X-Trail,2023,Offroader / SUV,Boz,2.5 L/184 a.g./Benzin,0 km,Avtomat,Bəli,"Vuruğu yoxdur, rənglənməyib",55 000 AZN,https://turbo.az/autos/7947885-nissan-x-trail
1,BMW,630,2005,Kabriolet,Qara,3.0 L/258 a.g./Benzin,12 706 km,Avtomat,Xeyr,"Vuruğu yoxdur, rənglənməyib",35 800 USD,https://turbo.az/autos/7785808-bmw-630
2,MG,ZS,2023,Offroader / SUV,Ağ,1.5 L/118 a.g./Benzin,0 km,Avtomat,Bəli,"Vuruğu yoxdur, rənglənməyib",21 470 USD,https://turbo.az/autos/7808576-mg-zs
3,BMW,M8,2022,Kupe,Qara,4.4 L/625 a.g./Benzin,12 000 km,Avtomat,Xeyr,"Vuruğu yoxdur, rənglənməyib",220 999 USD,https://turbo.az/autos/7596985-bmw-m8
4,Hyundai,Santa Fe,2013,Offroader / SUV,Ağ,2.4 L/175 a.g./Benzin,158 000 km,Avtomat,Xeyr,"Vuruğu yoxdur, rənglənməyib",36 300 AZN,https://turbo.az/autos/7919450-hyundai-santa-fe
5,Cadillac,CT6,2017,Sedan,Qara,2.0 L/269 a.g./Benzin,35 000 km,Avtomat,Xeyr,"Vuruğu yoxdur, rənglənməyib",30 500 USD,https://turbo.az/autos/7920012-cadillac-ct6
6,Audi,A4,2014,Sedan,Qara,2.0 L/225 a.g./Benzin,201 000 km,Avtomat,Xeyr,"Vuruğu yoxdur, rənglənməyib",24 300 AZN,https://turbo.az/autos/7779016-audi-a4
7,Kia,Rio,2008,Hetçbek,Qırmızı,1.4 L/97 a.g./Benzin,350 000 km,Mexaniki,Xeyr,"Vuruğu yoxdur, rənglənməyib",10 000 AZN,https://turbo.az/autos/7959876-kia-rio
8,Mercedes,GLS 400,2020,Offroader / SUV,Qara,2.9 L/330 a.g./Dizel,18 000 km,Avtomat,Xeyr,"Vuruğu yoxdur, rənglənməyib",112 000 USD,https://turbo.az/autos/7838008-mercedes-gls-400
9,Kia,Sorento,2018,Offroader / SUV,Yaş Asfalt,2.0 L/184 a.g./Dizel,135 000 km,Avtomat,Xeyr,"Vuruğu yoxdur, rənglənməyib",41 850 AZN,https://turbo.az/autos/7936006-kia-sorento


In [13]:
auto_data.tail()

,Marka,Model,Buraxılış ili,Ban növü,Rəng,Mühərrik,Yürüş,Sürətlər qutusu,Yeni,Vəziyyəti,Qiymət,Link
1045,Opel,Vectra,1998,Sedan,Qızılı,2.0 L/136 a.g./Benzin,574 246 km,Avtomat,Xeyr,NaN,5 900 AZN,https://turbo.az/autos/7963463-opel-vectra
1046,Opel,Astra,2006,Hetçbek,Tünd qırmızı,1.4 L/90 a.g./Benzin,259 282 km,Mexaniki,Xeyr,NaN,12 800 AZN,https://turbo.az/autos/7963559-opel-astra
1047,Toyota,RAV4,2021,Offroader / SUV,Boz,2.5 L/218 a.g./Hibrid,18 000 km,Avtomat,Xeyr,NaN,44 000 USD,https://turbo.az/autos/7963564-toyota-rav4
1048,Ford,Transit,2008,Furqon,Gümüşü,2.4 L/140 a.g./Dizel,176 000 km,Mexaniki,Xeyr,NaN,19 800 AZN,https://turbo.az/autos/7953753-ford-transit
1049,Lexus,GS 300,2006,Sedan,Göy,3.0 L/249 a.g./Benzin,195 000 km,Avtomat,Xeyr,NaN,15 800 AZN,https://turbo.az/autos/7914349-lexus-gs-300


In [16]:
auto_data.to_excel("auto_data.xlsx")